In [1]:
import os
import numpy as np

import CAS.dmrghandler.src.dmrghandler as dmrghandler
from CAS.dmrghandler.src.dmrghandler import dmrg_calc_prepare
import CAS_Cropping.ferm_utils as feru
from pathlib import Path
import openfermion as of
import pyscf.tools.fcidump
import pyscf
import pandas as pd
import os
import xlsxwriter

In [2]:
writer = pd.ExcelWriter("nearness_to_hubbard_may8_5pm.xlsx", engine='xlsxwriter')

In [ ]:
fcidump_path ="fcidumps_solvable_LPBLISS"
column_names = ["FCIDUMP file", "GE"]
data_to_be_added = []
for filename in os.listdir(fcidump_path):
    print(f"Current file: {filename}")
    (
    one_body_tensor,
    two_body_tensor,
    nuc_rep_energy,
    num_orbitals,
    num_spin_orbitals,
    num_electrons,
    two_S,
    two_Sz,
    orb_sym,
    extra_attributes,
    ) = dmrg_calc_prepare.load_tensors_from_fcidump(data_file_path=Path(fcidump_path)/Path(filename), molpro_orbsym_convention=True)
    ferm_op = feru.get_ferm_op(two_body_tensor) + feru.get_ferm_op(one_body_tensor)
    GE = of.get_ground_state(of.get_sparse_operator(ferm_op))[0]
    print(GE)
    data_to_be_added.append([filename, GE])

Current file: fcidump.61_3_15_af_{'default'_'cc-pVTZ''Mn'_'lanl2tz'}_tbt_combined_solvable
Parsing fcidumps_solvable_LPBLISS/fcidump.61_3_15_af_{'default'_'cc-pVTZ''Mn'_'lanl2tz'}_tbt_combined_solvable


In [ ]:
df_1 = pd.DataFrame()
#Iteratively add data to new columns
for column_name, column_data in zip(column_names, data_to_be_added):
    df_1[column_name] = column_data

df_1.to_excel(writer, sheet_name="Hubbard distance result", index=False)